In [1]:
import requests
import re
from multiprocessing import Process, Manager

MATCHING = (
    ('Python', '(p|P)ython'),
    ('Ruby', '(r|R)uby'),
    ('JavaScript', 'js|(J|j)ava(s|S)cript'),
    ('NodeJS', 'node(\.?)(?:\js|JS)'),
    ('Java', '(j|J)ava[^(S|s)cript]'),
    ('Objective-C', 'Obj(ective?)(?:\ |-)(C|c)'),
)

In [2]:
def get_story(story_id, stories):
    url = 'https://hacker-news.firebaseio.com/v0/item/%d.json' % story_id
    resp = requests.get(url)
    stories.append(resp.json())

In [3]:
def get_top_stories():
    manager = Manager()
    stories = manager.list()
    url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    ids = requests.get(url)
    processes = [Process(target=get_story, args=(sid, stories))
                 for sid in ids.json()[:40]]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    return stories

In [4]:
def count_languages():
    stories = get_top_stories()
    final_tallies = {}
    for s in stories:
        long_string = u'{} {}'.format(s.get('title'), s.get('url'))
        for language, regex in dict(MATCHING).items():
            if re.search(regex, long_string):
                if language not in final_tallies.keys():
                    final_tallies[language] = {
                        'score': s.get('score'),
                        'descendants': s.get('descendants')}
                else:
                    final_tallies[language]['score'] += s.get('score')
                    final_tallies[language][
                        'descendants'] += s.get('descendants')
    return final_tallies

In [5]:
count_languages()

{'JavaScript': {'descendants': 54, 'score': 88},
 'Python': {'descendants': 1, 'score': 28},
 'Ruby': {'descendants': 11, 'score': 59}}